In [13]:
## basic libraries → data exploration
import numpy as np
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt
import os
# from google.colab import drive

from tensorflow.keras.applications.resnet import ResNet152
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, BatchNormalization
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model

In [14]:
# drive.mount('content')

In [15]:
# defining data path
train_path = '../stanford_car_datasets/train/'
test_path = '../stanford_car_datasets/test/'

In [16]:
## loading the data on drive
# df_test_labels = pd.read_csv('content/MyDrive/Thesis_Smart_parking_222/stanford_car/testing_labels.csv')
# train_car = glob("/content/MyDrive/Thesis_Smart_parking_222/stanford_car/train/*/*")
# test_car = glob("/content/MyDrive/Thesis_Smart_parking_222/stanford_car/test/*/*")

## loading the data on local
df_test_labels = pd.read_csv('../stanford_car_datasets/testing_labels.csv')
train_car = glob(f"{train_path}*/*")
test_car = glob(f"{test_path}*/*")

In [17]:
class_names = list(df_test_labels["Cars"].unique())
class_names

['Acura Integra Type R 2001',
 'Acura RL Sedan 2012',
 'Acura TL Sedan 2012',
 'Acura TL Type-S 2008',
 'Acura TSX Sedan 2012',
 'Acura ZDX Hatchback 2012',
 'AM General Hummer SUV 2000',
 'Aston Martin V8 Vantage Convertible 2012',
 'Aston Martin V8 Vantage Coupe 2012',
 'Aston Martin Virage Convertible 2012',
 'Aston Martin Virage Coupe 2012',
 'Audi 100 Sedan 1994',
 'Audi 100 Wagon 1994',
 'Audi A5 Coupe 2012',
 'Audi R8 Coupe 2012',
 'Audi RS 4 Convertible 2008',
 'Audi S4 Sedan 2007',
 'Audi S4 Sedan 2012',
 'Audi S5 Convertible 2012',
 'Audi S5 Coupe 2012',
 'Audi S6 Sedan 2011',
 'Audi TT Hatchback 2011',
 'Audi TT RS Coupe 2012',
 'Audi TTS Coupe 2012',
 'Audi V8 Sedan 1994',
 'Bentley Arnage Sedan 2009',
 'Bentley Continental Flying Spur Sedan 2007',
 'Bentley Continental GT Coupe 2007',
 'Bentley Continental GT Coupe 2012',
 'Bentley Continental Supersports Conv. Convertible 2012',
 'Bentley Mulsanne Sedan 2011',
 'BMW 1 Series Convertible 2012',
 'BMW 1 Series Coupe 2012',


In [18]:
## setting up some parameters for data augmentation
img_width, img_height = 224, 224
num_channels = 3
train_samples = len(train_car)
test_samples = len(test_car)
num_of_class_names = len(class_names)
batch_size = 16
epochs = 50

In [19]:
## performing augmentation on the training data
train_datagen = ImageDataGenerator(
  zoom_range=0.2,
  rescale=1./255,
  rotation_range = 20,
  horizontal_flip=True
)

valid_datagen = ImageDataGenerator()

In [20]:
## converting data to a tf.data.Dataset object
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

valid_generator = valid_datagen.flow_from_directory(
    test_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


# Build ResNet152 Model

In [21]:
## load the ResNet152 model
model = ResNet152(
  include_top=False,
  weights='imagenet',
  input_tensor=None,
  input_shape=(img_width, img_height, 3),
  classes=num_of_class_names,
)

In [22]:
## add some extra layers
x = model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dense(
  units=1024,
  activation='relu',
  kernel_regularizer=regularizers.l2(0.01), 
  kernel_initializer='random_uniform',
  bias_initializer='zeros'
)(x)
x = Dense(
  units=1024,
  activation='relu',
  kernel_regularizer=regularizers.l2(0.01), 
  kernel_initializer='random_uniform',
  bias_initializer='zeros'
)(x)
output = Dense(units=num_of_class_names, activation='softmax')(x)

## create the extended model
tf_model = Model(inputs=model.input, outputs=output)

# freeze all layers excepts batchnorm layers
for layer in model.layers:
    if isinstance(layer, BatchNormalization):
        layer.trainable = True
    else:
        layer.trainable = False


In [23]:
model.summary()

Model: "resnet152"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                          

In [24]:
## compile the model, define optimizer and the loss function
opt = Adam(learning_rate=0.01)

tf_model.compile(
  loss='categorical_crossentropy',
  optimizer=opt, metrics=['accuracy']
)

In [12]:
checkpoint_path = "../model_resnet152/cp_resnet152_model.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = ModelCheckpoint(
  filepath=checkpoint_path,
  save_weights_only=True,
  verbose=1
)

In [66]:
# tf_model.load_weights(checkpoint_path)

In [ ]:
training_history = tf_model.fit(
  train_generator,
  steps_per_epoch=len(train_generator),
  validation_data=valid_generator,
  validation_steps=(len(valid_generator) / batch_size),
  epochs=epochs,
  callbacks=[cp_callback]
)

In [ ]:
accuracy = training_history.history['accuracy']
val_accuracy = training_history.history['val_accuracy']

loss = training_history.history['loss']
val_loss = training_history.history['val_loss']

epochs_range = range(20)

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, accuracy, label='Training Accuracy')
plt.plot(epochs_range, val_accuracy, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()